-> Chargement des librairies et fonctions

// Ici je retest tout avec les 10 topic majoritaire par et le titre

In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
my_local_drive = "/content/drive/MyDrive/projet_ML/Projet_MachineLearning"
# fonctions utilities (affichage, confusion, etc.)
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# fonctions utilities (affichage, confusion, etc.)
from Fonction.MyNLPUtilities import *
# fonctions utilities (fonction de clean, import etc etc)
from Fonction.myFonction import *

from Fonction.AllModels import *
# librairies de gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import corpora
from gensim import models

import re
import spacy
import gensim
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import gensim
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import corpora
from gensim import models
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
#nlp = spacy.load('en', disable=['parser', 'ner'])


data_brute = pd.read_csv('./Data_equilibre/MyData_Sprint1et2.csv', sep=",")
# data_brute = data_brute.drop_duplicates()
print(data_brute['our rating'].value_counts())


def MyCleanTextsforLDA(texts,
                       min_count=1, # nombre d'apparitions minimale pour un bigram
                       threshold=2,
                       no_below=1, # nombre minimum d'apparitions pour être dans le dictionnaire
                       no_above=0.5, # pourcentage maximal (sur la taille totale du corpus) pour filtrer
                       stop_words=stop_words
                       ):

    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    sentences=texts.copy()

    # suppression des caractères spéciaux
    sentences = [re.sub(r'[^\w\s]', ' ', str(sentence)) for sentence in sentences]
    # suppression de tous les caractères uniques
    sentences = [re.sub(r'\s+[a-zA-Z]\s+', ' ', str(sentence)) for sentence in sentences]
    # substitution des espaces multiples par un seul espace
    sentences = [re.sub(r'\s+', ' ', str(sentence), flags=re.I) for sentence in sentences]

    # conversion en minuscule et split des mots dans les textes
    sentences = [sentence.lower().split() for sentence in sentences]

    # utilisation de spacy pour ne retenir que les allowed_postags
    texts_out = []
    for sent in sentences:
        if len(sent) < (nlp.max_length): # si le texte est trop grand
            doc = nlp(" ".join(sent))
            texts_out.append(" ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))
        else:
            texts_out.append(sent)
    sentences=texts_out

    # suppression des stopwords
    words =[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in sentences]


    # recherche des bigrammes
    bigram = Phrases(words, min_count, threshold,delimiter=' ')
    bigram_phraser = Phraser(bigram)


    # sauvergarde des tokens et des bigrammes
    bigram_token = []
    for sent in words:
        bigram_token.append(bigram_phraser[sent])


    # creation du vocabulaire
    dictionary = gensim.corpora.Dictionary(bigram_token)


    # il est possible de filtrer des mots en fonction de leur occurrence d'apparitions
    #dictionary.filter_extremes(no_below, no_above)
    # et de compacter le dictionnaire
    # dictionary.compactify()
    corpus = [dictionary.doc2bow(text) for text in bigram_token]

    # recuperaction du tfidf plutôt que uniquement le bag of words
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]

    return corpus, corpus_tfidf, dictionary, bigram_token



def get_best_coherence_values(corpus, dictionary, listtokens, start=5, stop=15, step=2):
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=num_topics,
                                               random_state=100,
                                               chunksize=100,
                                               passes=10,
                                               workers=20,###
                                               eval_every=1,###
                                               alpha=0.01, ###
                                               eta=0.01,##
                                               per_word_topics=True)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=listtokens, dictionary=dictionary, coherence='c_v')
        model_list.append(lda_model)
        coherence_values.append(coherence_model_lda.get_coherence())
    return model_list, coherence_values







def format_topics_sentences(ldamodel, corpus, texts):
    # Initialisation du dataframe de sortie
    sent_topics_df = pd.DataFrame()

    # Recherche le topic dominant pour chaque document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list

        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Donne le topic dominant, le pourcentage de contribution
        # et les mots clés pour chaque document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => topic dominant
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['topic_dominant', 'pourcentage_contrib', 'topic_keywords']

    # Ajout du texte original à la fin de la sortie
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


stop = stopwords.words('english')


# enrichissement des stopwords 
stop.extend(['always','try','go','get','say','make','would','really',
                  'like','came','got'])





# ce code est inspiré de 
# https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
def compute_coherence_values(corpus, dictionary, listtokens, k, alpha, eta):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=eta,
                                           workers=7,###
                                           eval_every=1,###
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=listtokens, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

def MyGridSearchLda (corpus,listtokens,dictionnary,nb_topics,alpha,beta,verbose=1):
        
    grid = {}
    model_results = {'topics': [],
                     'alpha': [],
                     'eta': [],
                     'coherence': []
                    }
    # iteration sur le nombre de topics
    for k in nb_topics:
        # iteration sur les valeurs d'alpha
        for a in alpha:
            # iteration sur les valeurs de eta
            for e in eta:
                # calcul du score de coherence
                cv = compute_coherence_values(corpus=corpus, 
                                              dictionary=dictionary,
                                              listtokens=listtokens,
                                              k=k, alpha=a, eta=e)
                if verbose==1:
                    print ('topics:', k, ' alpha: %0.3f  eta: %0.3f  coherence: %0.3f'%(a,e,cv))
                        
                # sauvegarde des résultats
                model_results['topics'].append(k)
                model_results['alpha'].append(a)
                model_results['eta'].append(e)
                model_results['coherence'].append(cv)
                  
    df_result=pd.DataFrame(model_results)
    df_result = df_result.sort_values('coherence',ascending=False)
    df_result.reset_index(drop=True, inplace=True)
    return df_result


data_brute.title_text = data_brute.title + data_brute.text


corpus, corpus_tfidf, dictionary, bigram_token=MyCleanTextsforLDA(data_brute.title_text,stop_words=stop_words)
# variation du nombre de topics
min_topics = 10
max_topics = 20
step_size = 2
nb_topics = range(min_topics, max_topics, step_size)
# variation d'alpha
alpha = list(np.arange(0.5, 2, 0.5))
# variation de eta
eta = list(np.arange(0.5, 2, 0.5))
print ("Extraction des meilleurs paramètres pour le corpus via BOW : ")
df_result=MyGridSearchLda (corpus,bigram_token,dictionary,nb_topics,alpha,eta,verbose=1)  
display (df_result)















# corpus_all, corpus_tfidf_all, dictionary_all, bigram_token_all=MyCleanTextsforLDA(data_brute.text,stop_words=stop)


# model_list, coherence_values = get_best_coherence_values(dictionary=dictionary_all,
#                                                          corpus=corpus_all,
#                                                          listtokens=bigram_token_all,
#                                                          start=5, stop=60, step=5)

# # affichage du graphe associé à la recherche du nombre de topics
# plt.figure(figsize=(10,5))
# x = range(5, 60, 5)
# plt.plot(x, coherence_values)
# plt.xlabel("Nombre de topics")
# plt.ylabel("Coherence ")
# #plt.legend(("Valeurs de cohérencescoherence_values"), loc='best')
# plt.show()


# num_topics=21 # nombre de topics
# num_words=50 # nombre de mots par topics


# lda_model_all = gensim.models.ldamulticore.LdaMulticore(
#     corpus=corpus_all,
#     num_topics=num_topics,
#     id2word=dictionary_all,
#     chunksize=100,
#     workers=7,
#     passes=10,
#     random_state=100,
#     eval_every = 1,
#     per_word_topics=True)


# print ("Affichage des ",num_topics," différents topics pour le corpus BOW :\n")
# for idx, topic in lda_model_all.print_topics(-1,num_words):
#     print('Topic: {} Word: {}'.format(idx, topic))




# df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_all, corpus=corpus_all, texts=data_brute.text)


# display(df_topic_sents_keywords)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/projet_ML/Projet_MachineLearning


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


false      210
true       210
mixture    210
other      210
Name: our rating, dtype: int64


<ipython-input-4-d18f5beeeaa5>:238: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_brute.title_text = data_brute.title + data_brute.text


Extraction des meilleurs paramètres pour le corpus via BOW : 
topics: 10  alpha: 0.500  eta: 0.500  coherence: 0.302
topics: 10  alpha: 0.500  eta: 1.000  coherence: 0.367
topics: 10  alpha: 0.500  eta: 1.500  coherence: 0.341
topics: 10  alpha: 1.000  eta: 0.500  coherence: 0.345
topics: 10  alpha: 1.000  eta: 1.000  coherence: 0.417
topics: 10  alpha: 1.000  eta: 1.500  coherence: 0.365
topics: 10  alpha: 1.500  eta: 0.500  coherence: 0.348
topics: 10  alpha: 1.500  eta: 1.000  coherence: 0.366
topics: 10  alpha: 1.500  eta: 1.500  coherence: 0.470
topics: 12  alpha: 0.500  eta: 0.500  coherence: 0.369
topics: 12  alpha: 0.500  eta: 1.000  coherence: 0.387
topics: 12  alpha: 0.500  eta: 1.500  coherence: 0.367
topics: 12  alpha: 1.000  eta: 0.500  coherence: 0.404
topics: 12  alpha: 1.000  eta: 1.000  coherence: 0.365
topics: 12  alpha: 1.000  eta: 1.500  coherence: 0.279
topics: 12  alpha: 1.500  eta: 0.500  coherence: 0.346
topics: 12  alpha: 1.500  eta: 1.000  coherence: 0.451
top

,topics,alpha,eta,coherence
0,16,1.5,1.5,0.515825
1,14,1.0,1.5,0.512542
2,16,0.5,1.5,0.494890
3,18,1.0,1.5,0.492084
4,16,1.5,1.0,0.486113
5,16,1.0,1.5,0.476774
6,10,1.5,1.5,0.470384
7,18,0.5,1.5,0.469245
8,12,1.5,1.0,0.451040
9,18,1.0,1.0,0.422019


In [ ]:
import sys
from google.colab import drive
drive.mount('/content/drive')
my_local_drive = "/content/drive/MyDrive/projet_ML/Projet_MachineLearning"
# fonctions utilities (affichage, confusion, etc.)
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive
%pwd

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# fonctions utilities (affichage, confusion, etc.)
from Fonction.MyNLPUtilities import *
# fonctions utilities (fonction de clean, import etc etc)
from Fonction.myFonction import *

from Fonction.AllModels import *
# librairies de gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import corpora
from gensim import models

import re
import spacy
import gensim
import string
import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
import gensim
from gensim.utils import simple_preprocess
from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim import corpora
from gensim import models
nltk.download('wordnet')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
#nlp = spacy.load('en', disable=['parser', 'ner'])


data_brute = pd.read_csv('./Data_equilibre/MyData_Sprint1et2.csv', sep=",")
# data_brute = data_brute.drop_duplicates()
print(data_brute['our rating'].value_counts())


def MyCleanTextsforLDA(texts,
                       min_count=1, # nombre d'apparitions minimale pour un bigram
                       threshold=2,
                       no_below=1, # nombre minimum d'apparitions pour être dans le dictionnaire
                       no_above=0.5, # pourcentage maximal (sur la taille totale du corpus) pour filtrer
                       stop_words=stop_words
                       ):

    allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    sentences=texts.copy()

    # suppression des caractères spéciaux
    sentences = [re.sub(r'[^\w\s]', ' ', str(sentence)) for sentence in sentences]
    # suppression de tous les caractères uniques
    sentences = [re.sub(r'\s+[a-zA-Z]\s+', ' ', str(sentence)) for sentence in sentences]
    # substitution des espaces multiples par un seul espace
    sentences = [re.sub(r'\s+', ' ', str(sentence), flags=re.I) for sentence in sentences]

    # conversion en minuscule et split des mots dans les textes
    sentences = [sentence.lower().split() for sentence in sentences]

    # utilisation de spacy pour ne retenir que les allowed_postags
    texts_out = []
    for sent in sentences:
        if len(sent) < (nlp.max_length): # si le texte est trop grand
            doc = nlp(" ".join(sent))
            texts_out.append(" ".join([token.lemma_ for token in doc if token.pos_ in allowed_postags]))
        else:
            texts_out.append(sent)
    sentences=texts_out

    # suppression des stopwords
    words =[[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in sentences]


    # recherche des bigrammes
    bigram = Phrases(words, min_count, threshold,delimiter=' ')
    bigram_phraser = Phraser(bigram)


    # sauvergarde des tokens et des bigrammes
    bigram_token = []
    for sent in words:
        bigram_token.append(bigram_phraser[sent])


    # creation du vocabulaire
    dictionary = gensim.corpora.Dictionary(bigram_token)


    # il est possible de filtrer des mots en fonction de leur occurrence d'apparitions
    #dictionary.filter_extremes(no_below, no_above)
    # et de compacter le dictionnaire
    # dictionary.compactify()
    corpus = [dictionary.doc2bow(text) for text in bigram_token]

    # recuperaction du tfidf plutôt que uniquement le bag of words
    tfidf = models.TfidfModel(corpus)
    corpus_tfidf = tfidf[corpus]

    return corpus, corpus_tfidf, dictionary, bigram_token



def get_best_coherence_values(corpus, dictionary, listtokens, start=5, stop=15, step=2):
    coherence_values = []
    model_list = []
    for num_topics in range(start, stop, step):
        lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=num_topics,
                                               random_state=100,
                                               chunksize=100,
                                               passes=10,
                                               workers=20,###
                                               eval_every=1,###
                                               alpha=0.01, ###
                                               eta=0.01,##
                                               per_word_topics=True)

        coherence_model_lda = CoherenceModel(model=lda_model, texts=listtokens, dictionary=dictionary, coherence='c_v')
        model_list.append(lda_model)
        coherence_values.append(coherence_model_lda.get_coherence())
    return model_list, coherence_values







def format_topics_sentences(ldamodel, corpus, texts):
    # Initialisation du dataframe de sortie
    sent_topics_df = pd.DataFrame()

    # Recherche le topic dominant pour chaque document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list

        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Donne le topic dominant, le pourcentage de contribution
        # et les mots clés pour chaque document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => topic dominant
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['topic_dominant', 'pourcentage_contrib', 'topic_keywords']

    # Ajout du texte original à la fin de la sortie
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


stop = stopwords.words('english')


# enrichissement des stopwords 
stop.extend(['always','try','go','get','say','make','would','really',
                  'like','came','got'])





# ce code est inspiré de 
# https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
def compute_coherence_values(corpus, dictionary, listtokens, k, alpha, eta):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=alpha,
                                           eta=eta,
                                           workers=7,###
                                           eval_every=1,###
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=listtokens, dictionary=dictionary, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

def MyGridSearchLda (corpus,listtokens,dictionnary,nb_topics,alpha,beta,verbose=1):
        
    grid = {}
    model_results = {'topics': [],
                     'alpha': [],
                     'eta': [],
                     'coherence': []
                    }
    # iteration sur le nombre de topics
    for k in nb_topics:
        # iteration sur les valeurs d'alpha
        for a in alpha:
            # iteration sur les valeurs de eta
            for e in eta:
                # calcul du score de coherence
                cv = compute_coherence_values(corpus=corpus, 
                                              dictionary=dictionary,
                                              listtokens=listtokens,
                                              k=k, alpha=a, eta=e)
                if verbose==1:
                    print ('topics:', k, ' alpha: %0.3f  eta: %0.3f  coherence: %0.3f'%(a,e,cv))
                        
                # sauvegarde des résultats
                model_results['topics'].append(k)
                model_results['alpha'].append(a)
                model_results['eta'].append(e)
                model_results['coherence'].append(cv)
                  
    df_result=pd.DataFrame(model_results)
    df_result = df_result.sort_values('coherence',ascending=False)
    df_result.reset_index(drop=True, inplace=True)
    return df_result


data_brute.title_text = data_brute.title + data_brute.text


corpus, corpus_tfidf, dictionary, bigram_token=MyCleanTextsforLDA(data_brute.title_text,stop_words=stop_words)
# variation du nombre de topics
min_topics = 14
max_topics = 20
step_size = 2
nb_topics = range(min_topics, max_topics, step_size)
# variation d'alpha
alpha = list(np.arange(1, 2.5, 0.3))
# variation de eta
eta = list(np.arange(1, 2.5, 0.3))
print ("Extraction des meilleurs paramètres pour le corpus via BOW : ")
df_result=MyGridSearchLda (corpus,bigram_token,dictionary,nb_topics,alpha,eta,verbose=1)  
display (df_result)







Mounted at /content/drive
/content/drive/MyDrive/projet_ML/Projet_MachineLearning


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


false      210
true       210
mixture    210
other      210
Name: our rating, dtype: int64


<ipython-input-1-da4e05460f6a>:238: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  data_brute.title_text = data_brute.title + data_brute.text


Extraction des meilleurs paramètres pour le corpus via BOW : 
topics: 14  alpha: 1.000  eta: 1.000  coherence: 0.432
topics: 14  alpha: 1.000  eta: 1.300  coherence: 0.324
topics: 14  alpha: 1.000  eta: 1.600  coherence: 0.425
topics: 14  alpha: 1.000  eta: 1.900  coherence: 0.413
topics: 14  alpha: 1.000  eta: 2.200  coherence: 0.376
topics: 14  alpha: 1.300  eta: 1.000  coherence: 0.467
topics: 14  alpha: 1.300  eta: 1.300  coherence: 0.448
topics: 14  alpha: 1.300  eta: 1.600  coherence: 0.451
topics: 14  alpha: 1.300  eta: 1.900  coherence: 0.464
topics: 14  alpha: 1.300  eta: 2.200  coherence: 0.477
topics: 14  alpha: 1.600  eta: 1.000  coherence: 0.431
topics: 14  alpha: 1.600  eta: 1.300  coherence: 0.515
topics: 14  alpha: 1.600  eta: 1.600  coherence: 0.390
topics: 14  alpha: 1.600  eta: 1.900  coherence: 0.396
topics: 14  alpha: 1.600  eta: 2.200  coherence: 0.484
topics: 14  alpha: 1.900  eta: 1.000  coherence: 0.342
topics: 14  alpha: 1.900  eta: 1.300  coherence: 0.372
top

In [ ]:

num_topics=20 # nombre de topics
num_words=30 # nombre de mots par topics


lda_model_all = gensim.models.ldamulticore.LdaMulticore(
    corpus=corpus_all,
    num_topics=num_topics,
    id2word=dictionary_all,
    chunksize=400,
    workers=7,
    passes=20,
    random_state=100,
    eval_every = 1,
    per_word_topics=True)


print ("Affichage des ",num_topics," différents topics pour le corpus BOW :\n")
for idx, topic in lda_model_all.print_topics(-1,num_words):
    print('Topic: {} Word: {}'.format(idx, topic))




df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model_all, corpus=corpus_all, texts=data_brute.text)


display(df_topic_sents_keywords)

Affichage des  20  différents topics pour le corpus BOW :

Topic: 0 Word: 0.004*"self employ" + 0.003*"national insurance" + 0.003*"sst" + 0.003*"also" + 0.002*"datum" + 0.002*"show" + 0.002*"great barrier" + 0.002*"reef" + 0.002*"program" + 0.002*"scholar" + 0.002*"cool" + 0.002*"aim" + 0.002*"include" + 0.002*"oc" + 0.002*"sea surface" + 0.002*"university" + 0.002*"alien" + 0.002*"voter roll" + 0.002*"climate sensitivity" + 0.001*"latitude" + 0.001*"state" + 0.001*"student" + 0.001*"hank" + 0.001*"pilf" + 0.001*"datum centre" + 0.001*"gift" + 0.001*"australian current" + 0.001*"take" + 0.001*"month" + 0.001*"report"
Topic: 1 Word: 0.003*"report" + 0.003*"neighborhood" + 0.003*"human right" + 0.003*"case" + 0.003*"salvadoran" + 0.003*"police" + 0.003*"include" + 0.003*"kill" + 0.003*"interview" + 0.003*"victim" + 0.003*"also" + 0.003*"authority" + 0.002*"even" + 0.002*"tell" + 0.002*"people" + 0.002*"gang member" + 0.002*"know" + 0.002*"accord" + 0.002*"gang" + 0.002*"country" + 0.002

,topic_dominant,pourcentage_contrib,topic_keywords,text
0,4,0.9947,"year, work, state, people, bill, report, time,...",Urgent action needed to save ‘dying’ high stre...
1,6,0.6176,"use, year, chick pizza, party work, also, incl...",Denying 2000 years of the Medieval Warm Period...
2,18,0.5094,"vaccine, people, study, also, report, covid, c...","Dr. V.A. Shiva Ayyadurai, M.I.T. PhD, the Inve..."
3,5,0.9168,"people, time, know, report, year, also, state,...","Special Operations SOF units including 4,000 ..."
4,2,0.7710,"year, also, report, time, country, government,...","Bernie Sanders, an independent, represents Ver..."
...,...,...,...,...
835,0,0.9923,"self employ, national insurance, sst, also, da...",Rises in National Insurance Contributions NICS...
836,18,0.7623,"vaccine, people, study, also, report, covid, c...",U.S. Government Finally Admits Marijuana Reall...
837,14,0.9980,"country, people, vaccine, time, child, report,...",The nosediving security situation in the count...
838,2,0.8098,"year, also, report, time, country, government,...",Coronavirus may be sexually transmitted and ca...


In [ ]:

print(df_topic_sents_keywords['topic_keywords'][837])
print(df_topic_sents_keywords['text'][837])

country, people, vaccine, time, child, report, use, covid vaccine, dog, show
The nosediving security situation in the country is increasingly putting President Muhammadu Buhari under pressure as prominent Nigerians, among whom are governors, former political office holders, clerics and Nobel Laureate, Professor Wole Soyinka, have started to speak out in condemnation of his alleged inaction. The past two weeks have particularly been traumatic for the country, with killings, abductions and attacks on communities and security agencies, especially as killing of police officers continues unabated. Within the period, bandits operating in Kaduna State invaded a private institution in the State, Greenfield University in Chikun Local Government Area, abducted more than 20 students and killed a staff during the attack. In a gruesome move, the bandits, who asked for N800 million ransom, have killed five of the students to show that they meant business. During the period also, scores of innocent N

In [ ]:

df_all=data_brute
# modification du dataframe pour intégrer les mots associés au topic dominant à chaque document
df_all['keywords']=df_topic_sents_keywords['topic_keywords']
display(df_all)

df_all.to_csv('./Data_equilibre/mydata_topic.csv', sep=',', index=False)

,public_id,text,title,our rating,ID,Unnamed: 0,keywords
0,ff204eb9,Urgent action needed to save ‘dying’ high stre...,Urgent action needed to save ‘dying’ high stre...,false,NaN,NaN,"year, work, state, people, bill, report, time,..."
1,97b3e15c,Denying 2000 years of the Medieval Warm Period...,Gov’t Seeks to Control ‘Disorderly’ Internet P...,false,NaN,NaN,"use, year, chick pizza, party work, also, incl..."
2,NaN,"Dr. V.A. Shiva Ayyadurai, M.I.T. PhD, the Inve...","Dr. Shiva at AZ Senate Hearing: Over 17,000 To...",false,104589037423075649302005675867157134816,NaN,"vaccine, people, study, also, report, covid, c..."
3,18d2b528,"Special Operations SOF units including 4,000 ...",BREAKING: US Military at the White House Arres...,false,NaN,NaN,"people, time, know, report, year, also, state,..."
4,31d75b45,"Bernie Sanders, an independent, represents Ver...",Neil Mackay's Big Read: Inside Scotland’s educ...,false,NaN,NaN,"year, also, report, time, country, government,..."
...,...,...,...,...,...,...,...
835,f2a01c39,Rises in National Insurance Contributions NICS...,Budget 2017: National Insurance rate rise crit...,other,NaN,NaN,"self employ, national insurance, sst, also, da..."
836,0399ea89,U.S. Government Finally Admits Marijuana Reall...,"Abe Lincoln Statue Vandalized With Poop, Paint...",other,NaN,NaN,"vaccine, people, study, also, report, covid, c..."
837,9f89b2ba,The nosediving security situation in the count...,No help to buy – the flagship government polic...,other,NaN,NaN,"country, people, vaccine, time, child, report,..."
838,866fa600,Coronavirus may be sexually transmitted and ca...,Universal Credit leaves working families worse...,other,NaN,NaN,"year, also, report, time, country, government,..."
